In [1]:
import tkinter as tk
import numpy as np

In [2]:
class Estado:
    def __init__(self, fila, columna, costo, padre):
        self.fila = fila
        self.columna = columna
        self.costo = costo
        self.padre = padre

    def __lt__(self, other):
       
        return (self.costo) < (other.costo)

In [3]:
import tkinter as tk
import numpy as np

board_size = 8

white_color = 'white'
black_color = 'black'
default_color = 'lightblue'
board = [[' ' for _ in range(board_size)] for _ in range(board_size)]
board[3][3] = board[4][4] = 'W'
board[3][4] = board[4][3] = 'B'
print(board)

[[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', 'W', 'B', ' ', ' ', ' '], [' ', ' ', ' ', 'B', 'W', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]


In [4]:
points_direction = {
    0: 'N',
    1: 'NE',
    2: 'E',
    3: 'SE',
    4: 'S',
    5: 'SO',
    6: 'O',
    7: 'NO'
}

directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
def verify_limiter(x,y):
    return (x>-1 and y>-1 and x<board_size and y <board_size)
def hay_posiciones_libres():
    copy_board = np.array(board)
    return np.any(copy_board == ' ')

def counts_records():
    count_W = sum(row.count('W') for row in board)
    count_B = sum(row.count('B') for row in board)
    if count_W > count_B:
        print("¡El jugador blanco (W) gana!")
    elif count_B > count_W:
        print("¡El jugador negro (B) gana!")
    else:
        print("¡Empate!")
def get_opponent(player):
    return 'W' if player == 'B' else 'B'

def es_casilla_valida(row, col, player):
    if board[row][col] != ' ':
        return False

    for dr, dc in directions:
        y, x = row + dr, col + dc
        if not (0 <= y < board_size) or not (0 <= x < board_size):
            continue

        if board[y][x] != get_opponent(player):
            continue

        while 0 <= y < board_size and 0 <= x < board_size and board[y][x] == get_opponent(player):
            y += dr
            x += dc

        if 0 <= y < board_size and 0 <= x < board_size and board[y][x] == player:
            return True

    return False 
def hay_jugadas_disponibles(player):
    for row in range(board_size):
        for col in range(board_size):
            if es_casilla_valida(row, col,player):
                return True
    return False
def show_map(table):
    for y in range(board_size):
        for x in range(board_size):
            print(table[y][x])

def check_mov_direction(row, col,player):
    
    pos=0
    direction_check=[]
    check =False
    
    if player == "black":
        openent= 'W'
        player_turn = 'B'
    else:
        openent= 'B'
        player_turn = 'W'
    for dr, dc in directions:
        y,x=row+dr,col+dc
        if verify_limiter(y,x):
            while (x >= 0 and x < board_size) and (y >= 0 and y < board_size) and board[y][x] == openent:
                if verify_limiter(y + dr, x + dc):
                    y, x = y + dr, x + dc
                else:
                    break
          
            if board[y][x]==player_turn and verify_limiter(y,x):
                direction_check.append(points_direction[pos])
                check=True
        pos+=1    
    return direction_check,check



def check_mov(row, col,dir, player):
    coor_repaint=[]
    coor_repaint.append((row, col)) 
    if player == "black":
        openent= 'W'
        player = 'B'
    else:
        player = 'W'
        openent= 'B'
    y,x=row, col
    print("La direccion",dir, "filas y col",row, col)
    if dir == 'N':
        y-=1
        while board[y][x] == openent:
            coor_repaint.append((y,x)) 
            if verify_limiter(x,y-1):
                y-=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y-1][last_x] != player :
            coor_repaint=[]
    elif dir == 'NE':
        y-=1
        x+=1
        while  board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x+1,y-1):
                y-=1
                x+=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y-1][last_x+1] != player :
            coor_repaint=[]
    elif dir == 'E':
        x+=1 
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x+1,y):
                x+=1 
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y][last_x+1] != player :
            coor_repaint=[]
    elif dir == 'SE':
        y+=1
        x+=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x+1,y+1):
                y+=1
                x+=1 
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y+1][last_x+1] != player :
            coor_repaint=[]
    elif dir == 'S':
        y+=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x,y+1):
                y+=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        
        if board[last_y+1][last_x] != player :
            coor_repaint=[]
    elif dir == 'SO':
        y+=1
        x-=1 
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x-1,y+1):
                y+=1
                x-=1 
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y+1][last_x-1] != player:
            coor_repaint=[]
    elif dir == 'O':
        x-=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x-1,y):
                x-=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y][last_x-1] != player :
            coor_repaint=[]
    elif dir == 'NO':
        y-=1
        x-=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x-1,y-1):
                y-=1
                x-=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y-1][last_x-1] != player :
            coor_repaint=[]
    else:
        pass       
    return coor_repaint

def coor_to_paint(row, col,direction_check, player):
    result=[]
    for dir in direction_check:
        res=check_mov(row, col,dir, player)
        if res != [] :
            result.append(res)
    return result

## Heuristicas

In [5]:
def h1(board):
    corner_weight = 25
    own_piece_weight = 1
    opponent_piece_weight = -1
    score = 0
    corners = [(0, 0), (0, board_size - 1), (board_size - 1, 0), (board_size - 1, board_size - 1)]
    for corner in corners:
        if board[corner[0]][corner[1]] == 'B':
            score += corner_weight
        elif board[corner[0]][corner[1]] == 'W':
            score -= corner_weight
    for i in range(board_size):
        for j in range(board_size):
            if board[i][j] == 'B':
                score += own_piece_weight
            elif board[i][j] == 'W':
                score -= opponent_piece_weight

    return score


def h2(board):
    edge_weight = 10
    player = 'B'

    score = 0
    for i in range(board_size):
        for j in range(board_size):
            if (i == 0 or i == board_size - 1 or j == 0 or j == board_size - 1) and board[i][j] == player:
                score += edge_weight
            elif (i == 0 or i == board_size - 1 or j == 0 or j == board_size - 1) and board[i][j] == 'W':
                score -= edge_weight
    return score

In [6]:
def utility(board, player):
    enemy = 'W' if player == 'B' else player
    count_player = sum(row.count(player) for row in board)
    count_opponent = sum(row.count(enemy) for row in board)
    return count_player - count_opponent
#cor_besto=[]
def min_max_alpha_beta(board_generation, depth, player):
    if player == "W":
        best_eval = float('-inf')
        best_move = None  
        new_board = [row[:] for row in board_generation]
        coors_repaints = coor_to_paint(r, c, check_mov_direction(r, c, 'W'), 'W')
        
        eval, pos = min_value_alpha_beta(new_board, depth - 1, get_opponent(player))
        print("Lo ultimo que devuelve", eval, type(eval))
        if eval > best_eval:
            best_eval = eval
            best_move = pos  
        return best_eval, best_move  

    else:
        best_eval = float('inf')
        best_move = None 
        for r in range(board_size):
            for c in range(board_size):
                if board[r][c] == ' ' and es_casilla_valida(r, c, 'B'):
                    new_board = [row[:] for row in board_generation]
                    new_board[r][c] = 'B'
                    coors_repaints = coor_to_paint(r, c, check_mov_direction(r, c, 'B'), 'B')
                    for coor_repaint in coors_repaints:
                        for y, x in coor_repaint:
                            new_board[y][x] = 'B'
                    eval, pos = max_value_alpha_beta(new_board, depth - 1)
                    if eval < best_eval:
                        best_eval = eval
                        best_move = (r, c)
        return best_eval, best_move



def min_value_alpha_beta(board_generation, depth, player, alpha=float('inf'), beta=float('inf')):
    print("Entramos a min")
    if depth == 0 or not state_game():
        return utility(board_generation, player='W')
    
    v = float('inf')
    cor_besto = None  
    for r in range(board_size):
        for c in range(board_size):
            if alpha >= beta:
                continue
            check_corrse, check = check_mov_direction(r, c, 'white')
            if board_generation[r][c] == ' ' and check:
                new_board = [row[:] for row in board_generation]
                coors_repaints = coor_to_paint(r, c, check_corrse, 'white')
                for coor_repaint in coors_repaints:
                    for y, x in coor_repaint:
                        new_board[y][x] = 'W'
                eval_max = max_value_alpha_beta(new_board, depth - 1, get_opponent(player), alpha, beta)
                if eval_max < v:
                    v = eval_max
                    cor_besto = r, c  
                    alpha = max(alpha, v)
    print("el tipo de dato es:", cor_besto)
    return v, cor_besto  


def max_value_alpha_beta(board_generation, depth, player, alpha=float('inf'), beta=float('inf')):
    print("Entramos a max")
    if depth == 0 or not state_game():
        return utility(board_generation, player='W')
    
    v = float('-inf')
    cor_besto = None  
    for r in range(board_size):
        for c in range(board_size):
            if alpha >= beta:
                continue
            check_corrse, check = check_mov_direction(r, c, 'black')
            if board_generation[r][c] == ' ' and check:
                new_board = [row[:] for row in board_generation]
                coors_repaints = coor_to_paint(r, c, check_corrse, 'black')
                for coor_repaint in coors_repaints:
                    for y, x in coor_repaint:
                        new_board[y][x] = 'B'
                eval_min = min_value_alpha_beta(new_board, depth - 1, get_opponent(player), alpha, beta)  
                if eval_min > v:
                    v = eval_min
                    cor_besto = r, c  
                    alpha = max(alpha, v)
    print("el tipo de dato es:", cor_besto)
    return v, cor_besto  


In [7]:
def min_max(board_generation, depth, player):
    if player == "W":
        best_eval = float('inf')
        best_move = None
        for r in range(board_size):
            for c in range(board_size):
                if board[r][c] == ' ' and es_casilla_valida(r, c, 'W'):
                    new_board = [row[:] for row in board_generation]
                    new_board[r][c] = 'W'
                    coors_repaints = coor_to_paint(r, c, check_mov_direction(r, c, 'W'), 'W')
                    for coor_repaint in coors_repaints:
                        for y, x in coor_repaint:
                            new_board[y][x] = 'W'
                    eval = min_value(new_board, depth - 1, get_opponent(player))
                    if eval < best_eval:
                        best_eval = eval
                        best_move = (r, c)
        return best_eval, best_move
    else:
        best_eval = float('-inf')
        best_move = None
        for r in range(board_size):
            for c in range(board_size):
                if board[r][c] == ' ' and es_casilla_valida(r, c, 'B'):
                    new_board = [row[:] for row in board_generation]
                    new_board[r][c] = 'B'
                    coors_repaints = coor_to_paint(r, c, check_mov_direction(r, c, 'B'), 'B')
                    for coor_repaint in coors_repaints:
                        for y, x in coor_repaint:
                            new_board[y][x] = 'B'
                    eval = max_value(new_board, depth - 1, get_opponent(player))
                    if eval > best_eval:
                        best_eval = eval
                        best_move = (r, c)
        return best_eval, best_move


def min_value(board_generation, depth, player):
    print("Entramos a min")
    if depth == 0 or not state_game():
        return h1(board_generation)
    
    min_eval = float('inf')
    for r in range(board_size):
        for c in range(board_size):
            check_corrse, check = check_mov_direction(r, c, 'white')
            if board_generation[r][c] == ' ' and check:
                new_board = [row[:] for row in board_generation]
                coors_repaints = coor_to_paint(r, c, check_corrse, 'white')
                for coor_repaint in coors_repaints:
                    for y, x in coor_repaint:
                        new_board[y][x] = 'W'
                eval = max_value(new_board, depth - 1, get_opponent(player))  
                min_eval = min(min_eval, eval)
    return min_eval

def max_value(board_generation, depth, player):
    print("Entramos a max")
    if depth == 0 or not state_game():
        return h1(board_generation)
    
    value = float('-inf')
    for r in range(board_size):
        for c in range(board_size):
            check_corrse, check = check_mov_direction(r, c, 'black')
            if board_generation[r][c] == ' ' and check:
                new_board = [row[:] for row in board_generation]
                coors_repaints = coor_to_paint(r, c, check_corrse, 'black')
                for coor_repaint in coors_repaints:
                    for y, x in coor_repaint:
                        new_board[y][x] = 'B'
                eval = min_value(new_board, depth - 1, get_opponent(player))  
                value = max(value, eval)
    return value


def crear_casilla(tablero, row, col, color=default_color):
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
    button.grid(row=row + 1, column=col + 1)
    return button

def cambiar_color():
    global current_color
    current_color = 'W' if current_color == 'B' else 'B'
    label_color.config(text=f'Color actual: {current_color}')

def imprimir_coordenadas(row, col):
    print(f"Posición seleccionada: ({row}, {col})")

root = tk.Tk()
root.title('Othello')

tablero = tk.Frame(root)
tablero.pack()


def state_game():
    if hay_posiciones_libres() and ( hay_jugadas_disponibles('W') and hay_jugadas_disponibles('B')):
        return True
    else:
        counts_records()
        root.quit() 


def draw_map():
    for i in range(board_size):
        tk.Label(tablero, text=str(i + 1), width=3).grid(row=i + 1, column=0)
        tk.Label(tablero, text=chr(ord('A') + i), width=3).grid(row=0, column=i + 1)

    for row in range(board_size):
        for col in range(board_size):
            if (row == 3 and col == 3) or (row == 4 and col == 4):
                crear_casilla(tablero, row, col, 'white')
            elif (row == 3 and col == 4) or (row == 4 and col == 3):
                crear_casilla(tablero, row, col, 'black')
            else:
                crear_casilla(tablero, row, col)


def pintar_casilla(row, col):
    imprimir_coordenadas(row, col)
    if row != None and col !=None:
        if board[row][col] == ' ' and state_game():
            color = white_color if current_color == 'W' else black_color
            direction_check, check = check_mov_direction(row, col, color)
            
            if check:
                coors_repaints = coor_to_paint(row, col, direction_check, color)
                
                for coor_repaint in coors_repaints:
                    for y, x in coor_repaint:
                        board[y][x] = current_color
                        button = crear_casilla(tablero, y, x, color)  
                        button.config(state='disabled')  
                
                if current_color != opponent:  
                    cambiar_color()
            
                if current_color == opponent and state_game():  
                    print("EL color", current_color)
                    
                    
                    copy_board = [row[:] for row in board]
                    best_eval, best_move = min_max(copy_board, 3, opponent)
                    print("mejor movimiento", best_move)
                    if best_move is not None:
                        print("EL color y salí de la IA", current_color)
                        r, c = best_move[0],best_move[1]
                        opp_color = white_color if opponent == 'W' else black_color  
                        direction_check, check = check_mov_direction(r, c, opp_color)
                        if check:
                            
                            coors_repaints = coor_to_paint(r, c, direction_check, opp_color)
                            
                            for coor_repaint in coors_repaints:
                                for y, x in coor_repaint:
                                    board[y][x] = opponent
                                    button = crear_casilla(tablero, y, x, opp_color)  
                                    button.config(state='disabled') 
                            cambiar_color() 
                print("EL color y salí de la IA", current_color)

def is_not_blackPlayer():
    copy_board = [row[:] for row in board]
    best_eval, best_move = min_max(copy_board, 3, opponent)
    print("mejor movimiento", best_move)
    if best_move is not None:
        print("EL color y salí de la IA", current_color)
        r, c = best_move
        opp_color = white_color if opponent == 'W' else black_color  
        direction_check, check = check_mov_direction(r, c, opp_color)
        if check:
                
            coors_repaints = coor_to_paint(r, c, direction_check, opp_color)
            
            for coor_repaint in coors_repaints:
                for y, x in coor_repaint:
                    board[y][x] = opponent
                    button = crear_casilla(tablero, y, x, opp_color)  
                    button.config(state='disabled')

       
            

draw_map()
print("Seleccione qué fichas quiere usar", "Escriba 'B' Para jugar con las negras y 'W' para jugar con las blancas")
current_color = input()
opponent = 'W' if current_color == 'B' else 'B'
first_player = 'B'
label_color = tk.Label(root, text=f'Color actual: {current_color}')
label_color.pack()
if(current_color != first_player):
    is_not_blackPlayer()
if current_color == 'B' or current_color == 'W':
    root.mainloop()
else:
    print("Opción no válida")

Seleccione qué fichas quiere usar Escriba 'B' Para jugar con las negras y 'W' para jugar con las blancas
Posición seleccionada: (2, 3)
La direccion SE filas y col 2 3
La direccion S filas y col 2 3
EL color W
La direccion ['SE'] filas y col 2 2
La direccion True filas y col 2 2
Entramos a min
La direccion S filas y col 2 4
Entramos a max
La direccion SE filas y col 1 2
Entramos a min
La direccion S filas y col 1 3
Entramos a min
La direccion SO filas y col 1 4
Entramos a min
La direccion SO filas y col 2 5
Entramos a min
La direccion NE filas y col 3 2
La direccion E filas y col 3 2
La direccion SE filas y col 3 2
Entramos a min
La direccion O filas y col 3 5
Entramos a min
La direccion NE filas y col 4 2
La direccion E filas y col 4 2
Entramos a min
La direccion O filas y col 4 5
La direccion NO filas y col 4 5
Entramos a min
La direccion NE filas y col 5 2
Entramos a min
La direccion N filas y col 5 3
Entramos a min
La direccion N filas y col 5 4
La direccion NO filas y col 5 4
Entra

In [8]:
print(board)

[['W', 'W', 'W', 'W', 'W', ' ', ' ', ' '], ['B', 'B', 'B', 'B', 'W', ' ', ' ', ' '], ['W', 'W', 'W', 'W', 'B', 'W', ' ', ' '], [' ', ' ', 'W', 'B', 'B', 'B', ' ', ' '], [' ', 'W', ' ', 'B', 'B', ' ', ' ', ' '], [' ', ' ', ' ', ' ', 'B', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]


In [9]:
board[2][3]

'W'